In [119]:
from peewee import *
from playhouse.sqlite_ext import *

# pragmas = [
#     ('journal_mode', 'wal'),
#     ('cache_size', -1024 * 32)]
# db = SqliteExtDatabase('blog.db', pragmas=pragmas)
db = SqliteExtDatabase('blog.db')


class Entry(FTSModel):
    content = TextField()
    # You may have any number of other fields, such as status,
    # timestamp, etc.

    class Meta:
        database = db
        
class Tag(Model):
    title = TextField()

    class Meta:
        database = db
        
class TagMap(Model):
    tag_id = ForeignKeyField(Tag, backref='map_tag')
    entry_id = ForeignKeyField(Entry, backref='map_entry')

    class Meta:
        database = db
    
def add_content(content):
    return Entry.create(content=content)
    

def get_tag(title):
    try:
        tag = Tag.get(Tag.title == title)
        print('exist - ', title, tag.title, tag.id)
    except:    
        tag = Tag.create(title=title)
        print('new - ', title, tag.title, tag.id)
    return tag
    
def add_tag(docid, title):
    tag = get_tag(title)
    print(tag)
    TagMap.create(tag_id=tag.id, entry_id=docid)
    
    
def find_entry(content):
    return [entry for entry in Entry.select().where(Entry.match(content)).order_by(Entry.rank()).dicts()]

def find_tag(title):
    return [tag for tag in Tag.select().where(Tag.title.startswith(title)).dicts()]

def find_tag_entry(tag_id):
    return [entry for entry in Entry.select().join(TagMap, on=TagMap.entry_id).where(TagMap.tag_id == tag_id).dicts()]
    
    
db.create_tables([Entry, Tag, TagMap])    

In [86]:
doc = add_content('김문근')
doc = add_content('김문근 천재')
doc = add_content('김문근 천재 인가요?')

In [87]:
find_entry('김문근')

[{'docid': 1, 'content': '김문근'},
 {'docid': 2, 'content': '김문근 천재'},
 {'docid': 3, 'content': '김문근 천재 인가요?'}]

In [124]:
find_tag('인')

[{'id': 1, 'title': '인물'}, {'id': 2, 'title': '인사'}]

In [122]:
add_tag(1, '인물')

new -  인물 인물 1
1


In [123]:
add_tag(1, '인사')
add_tag(2, '인사')

new -  인사 인사 2
2
exist -  인사 인사 2
2


In [126]:
find_tag_entry(1)

[{'docid': 1, 'content': '김문근'}]